In [22]:
import requests
import json


In [44]:
requests.delete("http://localhost:9200/_ltr/")

<Response [200]>

In [45]:
requests.put("http://localhost:9200/_ltr/")

<Response [200]>

In [46]:
feature_set = {
   "featureset": {
      "features": [
         {
            "name": "title_bm25",
            "params": [
               "keywords"
            ],
            "template": {
                     "match": {
                        "title": "{{keywords}}"
                     }
               }
         },
         {
            "name": "overview_bm25",
            "params": [
               "keywords"
            ],
            "template": {
                     "match": {
                        "overview": "{{keywords}}"
                     }
               }
         }
      ]
   },
     "validation": {
      "index": "tmdb",
      "params": {
         "keywords": "rambo"
      }
   }
}

requests.put('http://localhost:9200/_ltr/_featureset/my_feature_set', 
             json=feature_set)

<Response [201]>

In [47]:
search = {
    "query": { 
    "match": {
        "title": "First Blood"
    }}
}

resp = requests.get('http://localhost:9200/tmdb/_search', json=search).json()
print(resp['hits']['hits'][0]['_source']['title'])

First Blood


In [48]:
search_with_log = {
  "query": {
    "bool": {
      "filter": [
        {
          "sltr": {
            "_name": "logged_features",
            "featureset": "my_feature_set",
            "params": {
              "keywords": "rambo"
            }
          }
        },
         {
          "terms": {
            "_id": [
              "7555","1370", "1369", "1368"
            ]
          }
        }
      ]
    }
  },
  "ext": {
    "ltr_log": {
      "log_specs": {
        "name": "ltr_features",
        "named_query": "logged_features"
      }
    }
  }
}

resp = requests.get('http://localhost:9200/tmdb/_search', json=search_with_log).json()
print(json.dumps(resp['hits']['hits'][0], indent=2))

{
  "_index": "tmdb",
  "_type": "movie",
  "_id": "1370",
  "_score": 0.0,
  "_source": {
    "id": "1370",
    "title": "Rambo III",
    "overview": "Combat has taken its toll on Rambo, but he's finally begun to find inner peace in a monastery. When Rambo's friend and mentor Col. Trautman asks for his help on a top secret mission to Afghanistan, Rambo declines but must reconsider when Trautman is captured.",
    "tagline": "The first was for himself. The second for his country. This time it's to save his friend.",
    "directors": [
      "Peter MacDonald"
    ],
    "cast": "Sylvester Stallone Richard Crenna Kurtwood Smith Spiros Foc\u00e1s Sasson Gabai Doudi Shoua Randy Raney Marcus Gilbert Alon Aboutboul Mahmoud Assadollahi Joseph Shiloach Harold Diamond Matti Seri Shaby Ben-Aroya Marc de Jonge",
    "genres": [
      "Action",
      "Adventure",
      "Thriller",
      "War"
    ],
    "release_date": "1988-05-24",
    "release_year": "1988",
    "vote_average": 5.6,
    "vote_co

In [49]:
model = """## LambdaMART
## No. of trees = 10
## No. of leaves = 10
## No. of threshold candidates = 256
## Learning rate = 0.1
## Stop early = 100

<ensemble>
	<tree id="1" weight="0.1">
		<split>
			<feature> 2 </feature>
			<threshold> 10.664251 </threshold>
			<split pos="left">
				<feature> 1 </feature>
				<threshold> 0.0 </threshold>
				<split pos="left">
					<output> -1.8305741548538208 </output>
				</split>
				<split pos="right">
					<feature> 2 </feature>
					<threshold> 9.502127 </threshold>
					<split pos="left">
						<feature> 1 </feature>
						<threshold> 7.0849166 </threshold>
						<split pos="left">
							<output> 0.23645669221878052 </output>
						</split>
						<split pos="right">
							<output> 1.7593677043914795 </output>
						</split>
					</split>
					<split pos="right">
						<output> 1.9719607830047607 </output>
					</split>
				</split>
			</split>
			<split pos="right">
				<feature> 2 </feature>
				<threshold> 0.0 </threshold>
				<split pos="left">
					<output> 1.3728954792022705 </output>
				</split>
				<split pos="right">
					<feature> 2 </feature>
					<threshold> 8.602512 </threshold>
					<split pos="left">
						<feature> 1 </feature>
						<threshold> 0.0 </threshold>
						<split pos="left">
							<feature> 2 </feature>
							<threshold> 13.815164 </threshold>
							<split pos="left">
								<output> 1.9401178359985352 </output>
							</split>
							<split pos="right">
								<output> 1.99532949924469 </output>
							</split>
						</split>
						<split pos="right">
							<feature> 1 </feature>
							<threshold> 11.085816 </threshold>
							<split pos="left">
								<output> 2.0 </output>
							</split>
							<split pos="right">
								<output> 1.99308180809021 </output>
							</split>
						</split>
					</split>
					<split pos="right">
						<output> 1.9870178699493408 </output>
					</split>
				</split>
			</split>
		</split>
	</tree>
</ensemble>
"""


create_model = {
  "model": {
     "name": "my_model",
     "model": {
         "type": "model/ranklib",
         "definition": model
    }
  }
}

requests.post('http://localhost:9200/_ltr/_featureset/my_feature_set/_createmodel',
              json=create_model).json()

{'_index': '.ltrstore',
 '_type': 'store',
 '_id': 'model-my_model',
 '_version': 1,
 'result': 'created',
 'forced_refresh': True,
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 1}

In [52]:
search = {
  "query": {
      "sltr": {
          "params": {
              "keywords": "rambo"
          },
          "model": "my_model"
      }
  }
}

In [53]:
for hit in resp['hits']['hits']:
    print(hit['_source']['title'])


Rambo III
First Blood
Rambo
Rambo: First Blood Part II
